In [12]:
SQLendpoint_ID = "" #OnyxToolLake sql endpoint
workspace_ID = ""   #OnyxMetadata-Prod



StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, 14, Finished, Available, Finished)

In [13]:
import json
import time
import struct
import sqlalchemy
import pyodbc
import notebookutils
import pandas as pd
from pyspark.sql import functions as fn
from pyspark.sql.functions import current_timestamp, expr
from datetime import datetime
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException

StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, 15, Finished, Available, Finished)

In [14]:
SQLEndpointsRefreshCalls="SQLEndpointsRefreshCalls"

StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, 16, Finished, Available, Finished)

### Create the Rest Client

In [15]:
#Instantiate the client
client = fabric.FabricRestClient()

StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, 17, Finished, Available, Finished)

### Check for a previous batch Id on the same SQL Endpoint Id

In [ ]:
batches=spark.read.format("delta").load(f'Tables/{SQLEndpointsRefreshCalls}')

batches = batches.filter(
    (batches.sqlEndpointId == SQLendpoint_ID) & 
    (batches.executionTime >= expr("current_timestamp() - interval 30 minutes"))
)

top_record = batches.orderBy(batches.executionTime.desc()).limit(1)

# if there is a batch, we will test the most recent one to confirm it's completed
if top_record.count() > 0:
    batch_id_value = top_record.collect()[0]["batchId"]
    # URL so we can get the status of the sync
    statusuri = f"/v1.0/myorg/lhdatamarts/{SQLendpoint_ID}/batches/{batch_id_value}"
    # turn response into object
    statusresponsedata = client.get(statusuri).json()
    # get the status of the check
    progressState = statusresponsedata["progressState"]
    if (progressState=="inProgress"):
        mssparkutils.notebook.exit(0)
    else:
        print(progressState)
else:
    print('no batch id record')


### It's ok to make a refresh request

In [ ]:
  
# URI for the call
uri = f"/v1.0/myorg/lhdatamarts/{SQLendpoint_ID}"
# This is the action, we want to take
payload = {"commands":[{"$type":"MetadataRefreshCommand"}]}
 
 
# Call the REST API
response = client.post(uri,json= payload)

StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, -1, Cancelled, , Cancelled)

### Let's Save the BatchId

In [ ]:

# return the response from json into an object we can get values from
data = json.loads(response.text)

# We just need this, we pass this to call to check the status
batchId = data["batchId"]

# Create a DataFrame for the new record
new_record = [(batchId, SQLendpoint_ID,  datetime.now())]
new_record_df = spark.createDataFrame(new_record, batches.schema)

display(new_record_df)

# Append the new record to the Delta table
new_record_df.write.format("delta").mode("append").save(f'Tables/{SQLEndpointsRefreshCalls}')

StatementMeta(, f9bac855-d0dc-4670-8a7e-8d337c9cf144, -1, Cancelled, , Cancelled)